In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Import other libraries if needed
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from urllib.parse import urlparse

In [2]:
df = pd.read_csv('train.csv')
df

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140399,235790,49490.txt,NaN,NaN,www.slavevoyages.org,NaN,0.0,NaN,1.000000,0.079963,...,NaN,0.0,1.0,23.0,3.0,6.0,NaN,12.0,NaN,1
140400,235791,NaN,https://www.greenmountainenergy.com,34.0,www.greenmountainenergy.com,NaN,0.0,com,1.000000,0.522907,...,1.0,NaN,1.0,26.0,NaN,NaN,169.0,15.0,40.0,1
140401,235792,NaN,https://www.leadcastingcall.com,30.0,www.leadcastingcall.com,23.0,NaN,NaN,1.000000,0.522907,...,0.0,0.0,1.0,25.0,NaN,NaN,87.0,1.0,93.0,1
140402,235794,NaN,https://www.fedarb.com,21.0,NaN,14.0,NaN,NaN,1.000000,0.522907,...,0.0,0.0,1.0,NaN,36.0,NaN,102.0,NaN,NaN,1


In [3]:
from sklearn.model_selection import train_test_split

# Save original training set
train_set_ori = df.copy()

# Split training set and validation set
train_set, val_set,  = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

In [ ]:
class HandleLength(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # No fitting process needed
        return self
    
    def transform(self, X):
        # Copy the DataFrame to avoid modifying the original
        X_transformed = X.copy()
        
        # Drop rows where URL is completely NaN
        X_transformed = X_transformed.dropna(subset=['URL'])
        
        # Impute Domain using URL if Domain is NaN
        def extract_domain(url):
            try:
                parsed_url = urlparse(url)
                return parsed_url.netloc if parsed_url.netloc else None
            except:
                return None

        X_transformed['Domain'] = X_transformed.apply(
            lambda row: extract_domain(row['URL']) if pd.isna(row['Domain']) else row['Domain'], axis=1
        )
        
        # Fill URLLength with the lengths of URL strings
        X_transformed['URLLength'] = X_transformed['URL'].apply(lambda x: len(str(x)) if pd.notna(x) else None)
        
        # Fill DomainLength with the lengths of Domain strings
        X_transformed['DomainLength'] = X_transformed['Domain'].apply(lambda x: len(str(x)) if pd.notna(x) else None)
        
        return X_transformed